# Data loading

Our pipeline starts with data loading step. Each batch run input files are delivered on location of `/Volumes/tantusdata_playground/default/bde-2023/input-partitioning/products-4096.parquet/`. The usual read & write time od 11s is considered too high, we need to lower it by a magniture of 2 or 3. 

## The goal
We want to decrease computing time in this notebook. Look into the UI and try to validate the performance problem. Then design and test a solution that modifies the input date _without changing any of its contents_ (so: pre-computing anything is not allowed).

The goal is not "5s is less than 7s", but to understand _why_ this change helped and how it can be fine-tuned.

In [0]:
import pyspark.sql.functions as F

# Input data path - this is the data that you need to load.
INPUT_PATH = "/Volumes/tantusdata_playground/default/bde-2023/input-partitioning/products-4096.parquet/"

# Volume path - make sure you have created a volume for yourself.
VOLUME_PATH = "/Volumes/tantusdata_playground/default/test-user-001"

IMMEDIATE_PATH = f"{VOLUME_PATH}/01-data-loading/staging/products.parquet"
WRITE_PATH = f"{VOLUME_PATH}/01-data-loading/products.parquet"

## Your own code:
Since we cannot interfere with the input, we are to emulate changing the partitioning there. **Write a step that**:
1. Loads the data from `INPUT_PATH`. 
2. Reorganizes the loaded data _without changing the contents_ (no pre-computing aggregates or anything similar). Try repartitioning the data.
3. Writes the data to `IMMEDIATE_PATH`. 

Afterwards the aggregation job should take less to complete. Try experimenting with different partitioning settings. You can use assertions at the end of the notebook to check job validity, but not the performance.

In [0]:
# FixMe: remove the line below and put your own processing step (job) here.
IMMEDIATE_PATH = INPUT_PATH

## The aggregation job
This job should complete faster solely due to your previous optimizations - do not modify it, rerun as-is to check the performance.

In [0]:
df_input = spark.read.parquet(IMMEDIATE_PATH)
df_transformed = (df_input
    .groupBy("unit")
    .agg(
        F.count_distinct("productID").alias("productsCount"), 
        F.avg("price").alias("averagePrice")
    )
)

df_transformed.write.mode("overwrite").parquet(WRITE_PATH)

## Assertions

In [0]:
assert(spark.read.parquet(IMMEDIATE_PATH).count() == 500_000)
assert(spark.read.parquet(WRITE_PATH).count() == 5)